In [1]:
#to perform linear and non linear models 
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [2]:
xtrain=r'C:/Users/MORA.SHIVARAMAKRISHN/Desktop/python_jupiter/assignment_data/x_train.csv'
x_train=pd.read_csv(xtrain)

ytrain=r'C:/Users/MORA.SHIVARAMAKRISHN/Desktop/python_jupiter/assignment_data/y_train.csv'
y_train=pd.read_csv(ytrain)

xtest=r'C:/Users/MORA.SHIVARAMAKRISHN/Desktop/python_jupiter/assignment_data/x_test.csv'
x_test=pd.read_csv(xtest)

ytest=r'C:/Users/MORA.SHIVARAMAKRISHN/Desktop/python_jupiter/assignment_data/y_test.csv'
y_test=pd.read_csv(ytest)


In [3]:
type(x_train)

pandas.core.frame.DataFrame

In [4]:
x_train.shape

(8124, 35)

In [5]:
type(y_train)

pandas.core.frame.DataFrame

In [6]:
y_train.shape

(8124, 1)

In [7]:
y_train

,y
0,0
1,0
2,0
3,0
4,0
...,...
8119,0
8120,0
8121,0
8122,0


In [8]:
y_train=pd.Series(y_train['y'])

In [9]:
type(y_train)

pandas.core.series.Series

In [10]:
y_train

0       0
1       0
2       0
3       0
4       0
       ..
8119    0
8120    0
8121    0
8122    0
8123    1
Name: y, Length: 8124, dtype: int64

In [11]:
type(y_test)

pandas.core.frame.DataFrame

In [12]:
y_test

,y
0,0
1,0
2,0
3,0
4,0
...,...
2026,0
2027,0
2028,1
2029,0


In [13]:
y_test=pd.Series(y_test['y'])

In [14]:
type(y_test)

pandas.core.series.Series

In [15]:
logr=LogisticRegression(penalty="l1",class_weight="balanced",random_state=2)

In [16]:
logr.fit(x_train,y_train)

C:\Users\MORA.SHIVARAMAKRISHN\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=None, penalty='l1',
                   random_state=2, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [17]:
roc_auc_score(y_test,logr.predict(x_test))

0.8995918649695628

In [18]:
prob_score=pd.Series(list(zip(*logr.predict_proba(x_train)))[1])

In [19]:
prob_score

0       0.040158
1       0.040054
2       0.014130
3       0.242985
4       0.040083
          ...   
8119    0.005901
8120    0.003018
8121    0.130037
8122    0.179023
8123    1.000000
Length: 8124, dtype: float64

In [20]:
cutoffs=np.linspace(0,1,100)

In [21]:
cutoffs

array([0.        , 0.01010101, 0.02020202, 0.03030303, 0.04040404,
       0.05050505, 0.06060606, 0.07070707, 0.08080808, 0.09090909,
       0.1010101 , 0.11111111, 0.12121212, 0.13131313, 0.14141414,
       0.15151515, 0.16161616, 0.17171717, 0.18181818, 0.19191919,
       0.2020202 , 0.21212121, 0.22222222, 0.23232323, 0.24242424,
       0.25252525, 0.26262626, 0.27272727, 0.28282828, 0.29292929,
       0.3030303 , 0.31313131, 0.32323232, 0.33333333, 0.34343434,
       0.35353535, 0.36363636, 0.37373737, 0.38383838, 0.39393939,
       0.4040404 , 0.41414141, 0.42424242, 0.43434343, 0.44444444,
       0.45454545, 0.46464646, 0.47474747, 0.48484848, 0.49494949,
       0.50505051, 0.51515152, 0.52525253, 0.53535354, 0.54545455,
       0.55555556, 0.56565657, 0.57575758, 0.58585859, 0.5959596 ,
       0.60606061, 0.61616162, 0.62626263, 0.63636364, 0.64646465,
       0.65656566, 0.66666667, 0.67676768, 0.68686869, 0.6969697 ,
       0.70707071, 0.71717172, 0.72727273, 0.73737374, 0.74747

In [22]:
KS_cut=[]
for cutoff in cutoffs:
    predicted=pd.Series([0]*len(y_train))
    predicted[prob_score>cutoff]=1
    df=pd.DataFrame(list(zip(y_train,predicted)),columns=["real","predicted"])
    TP=len(df[(df["real"]==1) &(df["predicted"]==1) ])
    FP=len(df[(df["real"]==0) &(df["predicted"]==1) ])
    TN=len(df[(df["real"]==0) &(df["predicted"]==0) ])
    FN=len(df[(df["real"]==1) &(df["predicted"]==0) ])
    P=TP+FN
    N=TN+FP
    KS=(TP/P)-(FP/N)
    KS_cut.append(KS)

cutoff_data=pd.DataFrame(list(zip(cutoffs,KS_cut)),columns=["cutoff","KS"])

KS_cutoff=cutoff_data[cutoff_data["KS"]==cutoff_data["KS"].max()]["cutoff"]

In [23]:
KS_cutoff

44    0.444444
Name: cutoff, dtype: float64

In [24]:
# Performance on test data
prob_score_test=pd.Series(list(zip(*logr.predict_proba(x_test)))[1])

predicted_test=pd.Series([0]*len(y_test))
predicted_test[prob_score_test>float(KS_cutoff)]=1

df_test=pd.DataFrame(list(zip(y_test,predicted_test)),columns=["real","predicted"])

k=pd.crosstab(df_test['real'],df_test["predicted"])
print('confusion matrix :\n \n ',k)
TN=k.iloc[0,0]
TP=k.iloc[1,1]
FP=k.iloc[0,1]
FN=k.iloc[1,0]
P=TP+FN
N=TN+FP

confusion matrix :
 
  predicted     0    1
real                
0          1646  161
1            26  198


In [25]:
# Accuracy of test
(TP+TN)/(P+N)

0.9079271294928607

In [26]:
# Sensitivity on test
TP/P

0.8839285714285714

In [27]:
#Specificity on test
TN/N

0.9109020475926951

In [28]:
TP/(TP+FP)  #F score is 0.67  for beta=1

0.5515320334261838

In [29]:
#f1 score
(2*0.55*0.88)/(0.55+0.88)

0.676923076923077

In [30]:
cutoffs=np.linspace(0.010,0.99,100)
def Fbeta_perf(beta,cutoffs,y_train,prob_score):
    FB_cut=[]
    for cutoff in cutoffs:
        predicted=pd.Series([0]*len(y_train))
        predicted[prob_score>cutoff]=1
        df=pd.DataFrame(list(zip(y_train,predicted)),columns=["real","predicted"])

        TP=len(df[(df["real"]==1) &(df["predicted"]==1) ])
        FP=len(df[(df["real"]==0) &(df["predicted"]==1) ])
        FN=len(df[(df["real"]==1) &(df["predicted"]==0) ])
        P=TP+FN
        
        
        Precision=TP/(TP+FP)
        Recall=TP/P
        FB=(1+beta**2)*Precision*Recall/((beta**2)*Precision+Recall)
        FB_cut.append(FB)

    cutoff_data=pd.DataFrame(list(zip(cutoffs,FB_cut)),columns=["cutoff","FB"])

    FB_cutoff=cutoff_data[cutoff_data["FB"]==cutoff_data["FB"].max()]["cutoff"]

    prob_score_test=pd.Series(list(zip(*logr.predict_proba(x_test)))[1])

    predicted_test=pd.Series([0]*len(y_test))
    predicted_test[prob_score_test>float(FB_cutoff)]=1

    df_test=pd.DataFrame(list(zip(y_test,predicted_test)),columns=["real","predicted"])

    k=pd.crosstab(df_test['real'],df_test["predicted"])
#     print('confusion matrix :\n \n ',k)
    TN=k.iloc[0,0]
    TP=k.iloc[1,1]
    FP=k.iloc[0,1]
    FN=k.iloc[1,0]
    P=TP+FN
    N=TN+FP
    print('For beta :',beta)
    print('Accuracy is :',(TP+TN)/(P+N))
    print('Sensitivity is :',(TP/P))
    print('Specificity is :',(TN/N))
    print('\n \n \n')

In [31]:
Fbeta_perf(0.5,cutoffs,y_train,prob_score)
Fbeta_perf(1,cutoffs,y_train,prob_score)
Fbeta_perf(2,cutoffs,y_train,prob_score)

For beta : 0.5
Accuracy is : 0.9399310684391925
Sensitivity is : 0.625
Specificity is : 0.9789706696181516

 
 

For beta : 1
Accuracy is : 0.9320531757754801
Sensitivity is : 0.7589285714285714
Specificity is : 0.9535141117874931

 
 

For beta : 2
Accuracy is : 0.92959133431807
Sensitivity is : 0.8348214285714286
Specificity is : 0.9413392363032651

 
 



In [32]:
#decision tree method
from sklearn import tree
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt


In [33]:
x_train.reset_index(drop=True,inplace=True)
y_train.reset_index(drop=True,inplace=True)

In [34]:
# creating a decision tree class object
dtree=tree.DecisionTreeClassifier(criterion="entropy",max_leaf_nodes=10,
                                  class_weight="balanced")

In [35]:
dtree.fit(x_train,y_train)

DecisionTreeClassifier(class_weight='balanced', criterion='entropy',
                       max_depth=None, max_features=None, max_leaf_nodes=10,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [36]:
predicted=dtree.predict(x_test)

df_test=pd.DataFrame(list(zip(y_test,predicted)),columns=["real","predicted"])

k=pd.crosstab(df_test['real'],df_test["predicted"])
k

predicted,0,1
real,,
0,1626,181
1,21,203


In [37]:
TP=k.iloc[1,1]
TN=k.iloc[0,0]
FP=k.iloc[0,1]
FN=k.iloc[1,0]
P=TP+FN
N=TN+FP


print('Accuracy is :',(TP+TN)/(P+N))
print('Sensitivity is :',TP/P)
print('Specificity is :',TN/N)


Accuracy is : 0.9005416051206302
Sensitivity is : 0.90625
Specificity is : 0.8998339789706696


In [38]:
#precision
203/(203+181)

0.5286458333333334

In [39]:
#f1 score
2*0.90625*0.52864/(0.90625+0.528645)

0.6677561772812645

In [40]:
max_nodes=[5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22]
max_nodes

[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]

In [41]:
beta=2
FB_avg=[]
for max_node  in max_nodes:
    mytree = tree.DecisionTreeClassifier(criterion="entropy",
                                         max_leaf_nodes=max_node,class_weight="balanced")

    # computing average RMSE across 10-fold cross validation
    kf = KFold( n_splits=10)
    FB_total = []
    for train, test in kf.split(x_train):
        mytree.fit(x_train.loc[train], y_train[train])
        p = mytree.predict(x_train.loc[test])
        df=pd.DataFrame(list(zip(y_train,p)),columns=["real","predicted"])
        TP=len(df[(df["real"]==1) &(df["predicted"]==1) ])
        FP=len(df[(df["real"]==0) &(df["predicted"]==1) ])
        TN=len(df[(df["real"]==0) &(df["predicted"]==0) ])
        FN=len(df[(df["real"]==1) &(df["predicted"]==0) ])
        P=TP+FN
        N=TN+FP
        Precision=TP/(TP+FP)
        Recall=TP/P
        FB=(1+beta**2)*Precision*Recall/((beta**2)*Precision+Recall)
        FB_total.extend([FB])
    FB_avg.extend([np.mean(FB_total)])
best_max_node=np.array(max_nodes)[FB_avg==max(FB_avg)][0]

print('max_node value with best F2 score is :',best_max_node)

max_node value with best F2 score is : 15


In [42]:
best_max_node

15

In [43]:
dtree=tree.DecisionTreeClassifier(criterion="entropy",
                                  max_leaf_nodes=best_max_node,class_weight="balanced")
dtree.fit(x_train,y_train)
predicted=dtree.predict(x_test)

df_test=pd.DataFrame(list(zip(y_test,predicted)),columns=["real","predicted"])

k=pd.crosstab(df_test['real'],df_test["predicted"])
print(k)

predicted     0    1
real                
0          1597  210
1            15  209


In [44]:
TP=k.iloc[1,1]
TN=k.iloc[0,0]
FP=k.iloc[0,1]
FN=k.iloc[1,0]
P=TP+FN
N=TN+FP

print(TP,TN,FP,FN)
print('Accuracy is :',(TP+TN)/(P+N))
Sensitivity=TP/P
print('Sensitivity is :',Sensitivity)
print('Specificity is :',TN/N)
precision=TP/(TP+FP)
print('precision is :',precision)


209 1597 210 15
Accuracy is : 0.8892171344165436
Sensitivity is : 0.9330357142857143
Specificity is : 0.8837852794687328
precision is : 0.4988066825775656


In [45]:
print('f1 score is :',(2*Sensitivity*precision)/(Sensitivity+precision))

f1 score is : 0.6500777604976672


In [46]:
#random forest
from sklearn.ensemble import RandomForestClassifier

In [47]:
import numpy as np

from time import time
from operator import itemgetter
from scipy.stats import randint as sp_randint

from sklearn.model_selection import RandomizedSearchCV


clf = RandomForestClassifier(verbose=1,n_jobs=-1)
def report(grid_scores, n_top=3):
    top_scores = sorted(grid_scores, key=itemgetter(1), reverse=True)[:n_top]
    
    for i, score in enumerate(top_scores):
        print("Model with rank: {0}".format(i + 1))
        print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
              score.mean_validation_score,
              np.std(score.cv_validation_scores)))
        print("Parameters: {0}".format(score.parameters))
        print("")
    

param_dist = {"n_estimators":[10,100,500,700],
              "max_depth": [3,5, None],
              "max_features": sp_randint(5, 11),
              "min_samples_split": sp_randint(5, 11),
              "min_samples_leaf": sp_randint(5, 11),
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}

n_iter_search = 100  
random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search)
random_search.fit(x_train, y_train)


C:\Users\MORA.SHIVARAMAKRISHN\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  78 tasks      | elapsed:    4.5s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    6.9s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    0.3s
[Parallel(n_jobs=4)]: Done 500 out of 500 | elapsed:    0.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 368 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed

[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.2s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.2s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:    0.1s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.2s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_

[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done 700 out of 700 | elapsed:    0.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 280 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done 700 out of 700 | elapsed:    6.1s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done 700 out of 700 | elapsed:    0.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.8s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 1

[Parallel(n_jobs=-1)]: Done 416 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    3.3s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done 500 out of 500 | elapsed:    0.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 416 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    3.4s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done 500 out of 500 | elapsed:    0.3s f

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    1.0s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    1.0s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend wit

...

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 416 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    2.5s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done 500 out of 500 | elapsed:    0.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 416 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    2.5s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 442 tasks      | ela

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 416 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 700 out of 700 | elapsed:    3.6s finished
[Parallel(n_jobs=4)]: Using backend

[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    0.3s
[Parallel(n_jobs=4)]: Done 500 out of 500 | elapsed:    0.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    1.5s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    0.3s
[Parallel(n_jobs=4)]: Done 500 out of 500 | elapsed:    0.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    1.4s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 442 tasks    

[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.1s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.1s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.1s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: 

RandomizedSearchCV(cv='warn', error_score='raise-deprecating',
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    n_estimators='warn',
                                                    n_jobs

In [48]:
print(random_search.cv_results_.keys())

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_bootstrap', 'param_criterion', 'param_max_depth', 'param_max_features', 'param_min_samples_leaf', 'param_min_samples_split', 'param_n_estimators', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score'])


In [49]:
random_search.cv_results_['rank_test_score']

array([ 52,  66,  32,  35,   1,  30,  11,  80,  47,  56,  25,  17,  24,
        68,  12,  73,  88,  66,  59,  83,   3,  59,  14,  12,  87,  57,
        20,  19,  77,  93,  43,  23,  49,  18,  82,  90,   7,  25,  28,
        38,  34,  35,   8,  71,  48,  44,  33,  76,  54,  75,  85,  96,
        81,   3,  21,  40,  37,  86,  89,  59, 100,  97,  62,   8,  94,
         5,  84,  29,  54,  95,  27,  41,   6,  10,  64,  51,  78,  50,
        57,  62,  16,  15,  69,  22,  73,  91,  64,  70,  46,   2,  53,
        98,  92,  31,  79,  99,  42,  39,  45,  72])

In [50]:
list(random_search.cv_results_['rank_test_score']).index(1)

4

In [51]:
random_search.cv_results_['params'][39]

{'bootstrap': True,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 5,
 'min_samples_leaf': 6,
 'min_samples_split': 10,
 'n_estimators': 10}

In [52]:
rf=RandomForestClassifier(n_estimators=500,verbose=1,criterion='entropy',min_samples_split=9,
                         bootstrap=False,max_depth=None,max_features=9,min_samples_leaf=5,
                          class_weight="balanced")

In [53]:
rf.fit(x_train,y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:   31.4s finished


RandomForestClassifier(bootstrap=False, class_weight='balanced',
                       criterion='entropy', max_depth=None, max_features=9,
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=5,
                       min_samples_split=9, min_weight_fraction_leaf=0.0,
                       n_estimators=500, n_jobs=None, oob_score=False,
                       random_state=None, verbose=1, warm_start=False)

In [54]:
predicted=rf.predict(x_test)

df_test=pd.DataFrame(list(zip(y_test,predicted)),columns=["real","predicted"])

k=pd.crosstab(df_test['real'],df_test["predicted"])
print(k)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


predicted     0    1
real                
0          1765   42
1            33  191


[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    0.1s finished


In [55]:
TP=k.iloc[1,1]
TN=k.iloc[0,0]
FP=k.iloc[0,1]
FN=k.iloc[1,0]
P=TP+FN
N=TN+FP

print(TP,TN,FP,FN)
print('Accuracy is :',(TP+TN)/(P+N))
Sensitivity=TP/P
print('Sensitivity is :',Sensitivity)
print('Specificity is :',TN/N)
precision=TP/(TP+FP)
print('precision is :',precision)
print('f1 score is :',(2*Sensitivity*precision)/(Sensitivity+precision))

191 1765 42 33
Accuracy is : 0.9630723781388478
Sensitivity is : 0.8526785714285714
Specificity is : 0.9767570558937465
precision is : 0.8197424892703863
f1 score is : 0.835886214442013


In [56]:
#gbm
import pandas as pd
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from xgboost.sklearn import XGBClassifier

In [57]:
gbm_params={'n_estimators':[50,100,200,500,700],
           'learning_rate': [0.01,.05,0.1,0.4,0.8,1],
            'max_depth':[1,2,3,4,5,6],
#             'min_samples_split':[2,5,10,20],
#             'min_samples_leaf':[2,5,10,20],
            'subsample':[0.5,0.8,1],
            'max_features':[5,10,15,20,30]
           }

In [58]:
gbm=GradientBoostingClassifier()

In [59]:
random_search=RandomizedSearchCV(gbm,scoring='roc_auc',param_distributions=gbm_params,
                                 cv=10,n_iter=10,
                                 n_jobs=-1)

In [60]:
random_search.fit(x_train,y_train)

RandomizedSearchCV(cv=10, error_score='raise-deprecating',
                   estimator=GradientBoostingClassifier(criterion='friedman_mse',
                                                        init=None,
                                                        learning_rate=0.1,
                                                        loss='deviance',
                                                        max_depth=3,
                                                        max_features=None,
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=1,
                                                        min_samples_split=2,
                                                        min_weight_fraction_leaf=0.0,
                    

In [61]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.5f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [62]:
report(random_search.cv_results_,5)

Model with rank: 1
Mean validation score: 0.992 (std: 0.00288)
Parameters: {'subsample': 0.8, 'n_estimators': 700, 'max_features': 10, 'max_depth': 5, 'learning_rate': 0.1}

Model with rank: 2
Mean validation score: 0.990 (std: 0.00233)
Parameters: {'subsample': 1, 'n_estimators': 700, 'max_features': 15, 'max_depth': 4, 'learning_rate': 0.05}

Model with rank: 3
Mean validation score: 0.987 (std: 0.00409)
Parameters: {'subsample': 0.8, 'n_estimators': 100, 'max_features': 10, 'max_depth': 5, 'learning_rate': 0.4}

Model with rank: 4
Mean validation score: 0.979 (std: 0.00732)
Parameters: {'subsample': 0.8, 'n_estimators': 50, 'max_features': 10, 'max_depth': 5, 'learning_rate': 0.05}

Model with rank: 5
Mean validation score: 0.974 (std: 0.00825)
Parameters: {'subsample': 1, 'n_estimators': 700, 'max_features': 15, 'max_depth': 2, 'learning_rate': 0.01}



In [63]:
predicted=random_search.predict(x_test)

df_test=pd.DataFrame(list(zip(y_test,predicted)),columns=["real","predicted"])

k=pd.crosstab(df_test['real'],df_test["predicted"])
print(k)

predicted     0    1
real                
0          1789   18
1            51  173


In [64]:
TP=k.iloc[1,1]
TN=k.iloc[0,0]
FP=k.iloc[0,1]
FN=k.iloc[1,0]
P=TP+FN
N=TN+FP

print(TP,TN,FP,FN)
print('Accuracy is :',(TP+TN)/(P+N))
Sensitivity=TP/P
print('Sensitivity is :',Sensitivity)
print('Specificity is :',TN/N)
precision=TP/(TP+FP)
print('precision is :',precision)
print('f1 score is :',(2*Sensitivity*precision)/(Sensitivity+precision))

173 1789 18 51
Accuracy is : 0.96602658788774
Sensitivity is : 0.7723214285714286
Specificity is : 0.9900387382401771
precision is : 0.9057591623036649
f1 score is : 0.8337349397590361


In [65]:
#xgboost
xgb_params = {  
                "learning_rate":[0.01,0.05,0.1,0.3,0.5],
                "gamma":[i/10.0 for i in range(0,5)],
                "max_depth": [2,3,4,5,6,7,8],
                "min_child_weight":[1,2,5,10],
                "max_delta_step":[0,1,2,5,10],
                "subsample":[i/10.0 for i in range(5,10)],
                "colsample_bytree":[i/10.0 for i in range(5,10)],
                "colsample_bylevel":[i/10.0 for i in range(5,10)],
                "reg_lambda":[1e-5, 1e-2, 0.1, 1, 100], 
                "reg_alpha":[1e-5, 1e-2, 0.1, 1, 100],
                "scale_pos_weight":[1,2,3,4,5,6,7,8,9],
                "n_estimators":[100,500,700,1000]
             }

In [66]:
xgb=XGBClassifier(objective='binary:logistic')

In [67]:
n_iter=10

random_search=RandomizedSearchCV(xgb,n_jobs=-1,cv=5,n_iter=n_iter,scoring='roc_auc',
                                 param_distributions=xgb_params)

In [68]:
random_search.fit(x_train,y_train)

C:\Users\MORA.SHIVARAMAKRISHN\Anaconda3\lib\site-packages\joblib\externals\loky\process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
                   estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                           colsample_bylevel=1,
                                           colsample_bynode=1,
                                           colsample_bytree=1, gamma=0,
                                           learning_rate=0.1, max_delta_step=0,
                                           max_depth=3, min_child_weight=1,
                                           missing=None, n_estimators=100,
                                           n_jobs=1, nthread=None,
                                           objective='binary:logistic',
                                           random_state=0, reg_alpha=0...
                                        'max_delta_step': [0, 1, 2, 5, 10],
                                        'max_depth': [2, 3, 4, 5, 6, 7, 8],
                                        'min_child_weight': [1, 2, 5, 

In [69]:
report(random_search.cv_results_,5)

Model with rank: 1
Mean validation score: 0.990 (std: 0.00110)
Parameters: {'subsample': 0.8, 'scale_pos_weight': 3, 'reg_lambda': 0.1, 'reg_alpha': 0.01, 'n_estimators': 700, 'min_child_weight': 1, 'max_depth': 5, 'max_delta_step': 5, 'learning_rate': 0.3, 'gamma': 0.3, 'colsample_bytree': 0.8, 'colsample_bylevel': 0.9}

Model with rank: 2
Mean validation score: 0.990 (std: 0.00196)
Parameters: {'subsample': 0.6, 'scale_pos_weight': 8, 'reg_lambda': 1, 'reg_alpha': 0.1, 'n_estimators': 500, 'min_child_weight': 1, 'max_depth': 8, 'max_delta_step': 0, 'learning_rate': 0.01, 'gamma': 0.3, 'colsample_bytree': 0.9, 'colsample_bylevel': 0.8}

Model with rank: 3
Mean validation score: 0.986 (std: 0.00198)
Parameters: {'subsample': 0.8, 'scale_pos_weight': 9, 'reg_lambda': 1, 'reg_alpha': 1e-05, 'n_estimators': 500, 'min_child_weight': 5, 'max_depth': 2, 'max_delta_step': 1, 'learning_rate': 0.1, 'gamma': 0.0, 'colsample_bytree': 0.6, 'colsample_bylevel': 0.8}

Model with rank: 4
Mean validat

In [79]:
predicted=random_search.predict(x_test)

df_test=pd.DataFrame(list(zip(y_test,predicted)),columns=["real","predicted"])

k=pd.crosstab(df_test['real'],df_test["predicted"])
print(k)

predicted     0    1
real                
0          1779   28
1            34  190


In [80]:
TP=k.iloc[1,1]
TN=k.iloc[0,0]
FP=k.iloc[0,1]
FN=k.iloc[1,0]
P=TP+FN
N=TN+FP

print(TP,TN,FP,FN)
print('Accuracy is :',(TP+TN)/(P+N))
Sensitivity=TP/P
print('Sensitivity is :',Sensitivity)
print('Specificity is :',TN/N)
precision=TP/(TP+FP)
print('precision is :',precision)
print('f1 score is :',(2*Sensitivity*precision)/(Sensitivity+precision))

190 1779 28 34
Accuracy is : 0.9694731659281143
Sensitivity is : 0.8482142857142857
Specificity is : 0.9845047039291643
precision is : 0.8715596330275229
f1 score is : 0.8597285067873303


In [16]:
#ANN
from sklearn.neural_network import MLPClassifier  #multi layer perceptron(nodes)
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import RandomizedSearchCV  #to select the best combination of hyper parammeters

In [17]:
help(MLPClassifier)

Help on class MLPClassifier in module sklearn.neural_network.multilayer_perceptron:

class MLPClassifier(BaseMultilayerPerceptron, sklearn.base.ClassifierMixin)
 |  MLPClassifier(hidden_layer_sizes=(100,), activation='relu', solver='adam', alpha=0.0001, batch_size='auto', learning_rate='constant', learning_rate_init=0.001, power_t=0.5, max_iter=200, shuffle=True, random_state=None, tol=0.0001, verbose=False, warm_start=False, momentum=0.9, nesterovs_momentum=True, early_stopping=False, validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08, n_iter_no_change=10)
 |  
 |  Multi-layer Perceptron classifier.
 |  
 |  This model optimizes the log-loss function using LBFGS or stochastic
 |  gradient descent.
 |  
 |  .. versionadded:: 0.18
 |  
 |  Parameters
 |  ----------
 |  hidden_layer_sizes : tuple, length = n_layers - 2, default (100,)
 |      The ith element represents the number of neurons in the ith
 |      hidden layer.
 |  
 |  activation : {'identity', 'logistic', 'tan

In [20]:
parameters={
'learning_rate': ["constant", "invscaling", "adaptive"],
'hidden_layer_sizes': [(5,10,5),(20,10),(10,20),(10,10,15)],  #5 nodes, 10 nodes and 5 nodes of 3 hidden layers
'alpha': [0.3,.1,.01],
'activation': [ "relu", 'logistic',"tanh"]
}

In [24]:
clf=MLPClassifier()

In [22]:
random_search=RandomizedSearchCV(clf,n_iter=30,cv=10,param_distributions=parameters,
                                 scoring='roc_auc',random_state=2,n_jobs=-1,verbose=20)

In [23]:
random_search.fit(x_train,y_train)

Fitting 10 folds for each of 30 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   17.1s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   17.4s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:   17.6s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:   18.3s
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   27.2s
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:   27.8s
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:   28.3s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:   28.4s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   34.0s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   35.8s
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:   36.5s
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:   38.7s
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:   39.0s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:   42.4s
[Parallel(n_jobs=-1)]: Done  15 tasks      | elapsed:   

[Parallel(n_jobs=-1)]: Done 132 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 133 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 134 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 135 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 136 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 137 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 139 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 140 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 141 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 142 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 143 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 144 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done 145 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done 147 tasks      | elapsed:  4.3min
[Paralle

[Parallel(n_jobs=-1)]: Done 265 tasks      | elapsed:  6.2min
[Parallel(n_jobs=-1)]: Done 266 tasks      | elapsed:  6.2min
[Parallel(n_jobs=-1)]: Done 267 tasks      | elapsed:  6.2min
[Parallel(n_jobs=-1)]: Done 268 tasks      | elapsed:  6.2min
[Parallel(n_jobs=-1)]: Done 269 tasks      | elapsed:  6.2min
[Parallel(n_jobs=-1)]: Done 270 tasks      | elapsed:  6.2min
[Parallel(n_jobs=-1)]: Done 271 tasks      | elapsed:  6.2min
[Parallel(n_jobs=-1)]: Done 272 tasks      | elapsed:  6.2min
[Parallel(n_jobs=-1)]: Done 273 tasks      | elapsed:  6.3min
[Parallel(n_jobs=-1)]: Done 274 tasks      | elapsed:  6.3min
[Parallel(n_jobs=-1)]: Done 275 tasks      | elapsed:  6.3min
[Parallel(n_jobs=-1)]: Done 276 tasks      | elapsed:  6.3min
[Parallel(n_jobs=-1)]: Done 277 tasks      | elapsed:  6.3min
[Parallel(n_jobs=-1)]: Done 278 tasks      | elapsed:  6.3min
[Parallel(n_jobs=-1)]: Done 279 tasks      | elapsed:  6.4min
[Parallel(n_jobs=-1)]: Done 280 tasks      | elapsed:  6.4min
[Paralle

RandomizedSearchCV(cv=10, error_score='raise-deprecating',
                   estimator=MLPClassifier(activation='relu', alpha=0.0001,
                                           batch_size='auto', beta_1=0.9,
                                           beta_2=0.999, early_stopping=False,
                                           epsilon=1e-08,
                                           hidden_layer_sizes=(100,),
                                           learning_rate='constant',
                                           learning_rate_init=0.001,
                                           max_iter=200, momentum=0.9,
                                           n_iter_no_change=10,
                                           nesterovs_momentum=True, power_t=0.5,
                                           ran...
                                           verbose=False, warm_start=False),
                   iid='warn', n_iter=30, n_jobs=-1,
                   param_distributions={'activatio

In [25]:
random_search.best_params_

{'learning_rate': 'invscaling',
 'hidden_layer_sizes': (20, 10),
 'alpha': 0.01,
 'activation': 'logistic'}

In [26]:
#{'learning_rate': 'invscaling',
 #'hidden_layer_sizes': (20, 30),
 #'alpha': 0.3,
 #'activation': 'relu'}


In [27]:
random_search.best_estimator_

MLPClassifier(activation='logistic', alpha=0.01, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(20, 10), learning_rate='invscaling',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=None, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [28]:
mlp=random_search.best_estimator_

In [29]:
mlp.fit(x_train,y_train)

MLPClassifier(activation='logistic', alpha=0.01, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(20, 10), learning_rate='invscaling',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=None, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [30]:
predicted=random_search.predict(x_test)

df_test=pd.DataFrame(list(zip(y_test,predicted)),columns=["real","predicted"])

k=pd.crosstab(df_test['real'],df_test["predicted"])
print(k)

predicted     0    1
real                
0          1758   49
1            59  165


In [31]:
TP=k.iloc[1,1]
TN=k.iloc[0,0]
FP=k.iloc[0,1]
FN=k.iloc[1,0]
P=TP+FN
N=TN+FP

print(TP,TN,FP,FN)
print('Accuracy is :',(TP+TN)/(P+N))
Sensitivity=TP/P
print('Sensitivity is :',Sensitivity)
print('Specificity is :',TN/N)
precision=TP/(TP+FP)
print('precision is :',precision)
print('f1 score is :',(2*Sensitivity*precision)/(Sensitivity+precision))

165 1758 49 59
Accuracy is : 0.946824224519941
Sensitivity is : 0.7366071428571429
Specificity is : 0.9728832318760376
precision is : 0.7710280373831776
f1 score is : 0.7534246575342466


In [ ]:
#2 iteration

In [32]:
from sklearn.model_selection import GridSearchCV

In [37]:
parameters={
'learning_rate': ["invscaling"],
'hidden_layer_sizes': [(5,10,5),(20,10),(10,20),(10,10,5)],  #5 nodes, 10 nodes and 5 nodes of 3 hidden layers
'alpha': [0.3,.1,.01],
'activation': [ "relu",'logistic',"tanh"]
}

In [38]:
clf2=MLPClassifier()

In [39]:
grid_search=GridSearchCV(clf2,cv=10,param_grid=parameters,scoring='roc_auc',verbose=2,n_jobs=-1)

In [40]:
grid_search.fit(x_train,y_train)

Fitting 10 folds for each of 36 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   30.7s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed:  5.9min finished


GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=MLPClassifier(activation='relu', alpha=0.0001,
                                     batch_size='auto', beta_1=0.9,
                                     beta_2=0.999, early_stopping=False,
                                     epsilon=1e-08, hidden_layer_sizes=(100,),
                                     learning_rate='constant',
                                     learning_rate_init=0.001, max_iter=200,
                                     momentum=0.9, n_iter_no_change=10,
                                     nesterovs_momentum=True, power_t=0.5,
                                     random_st...
                                     solver='adam', tol=0.0001,
                                     validation_fraction=0.1, verbose=False,
                                     warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid={'activation': ['relu', 'logistic', 'tanh'],
                  

In [41]:
grid_search.best_params_

{'activation': 'logistic',
 'alpha': 0.1,
 'hidden_layer_sizes': (20, 10),
 'learning_rate': 'invscaling'}

In [42]:
#{'activation': 'relu',
# 'alpha': 0.1,
# 'hidden_layer_sizes': (10, 10, 5),
# 'learning_rate': 'invscaling'}

In [43]:
grid_search.best_estimator_

MLPClassifier(activation='logistic', alpha=0.1, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(20, 10), learning_rate='invscaling',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=None, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [44]:
mlp2=grid_search.best_estimator_

In [45]:
mlp2.fit(x_train,y_train)

MLPClassifier(activation='logistic', alpha=0.1, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(20, 10), learning_rate='invscaling',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=None, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [46]:
predicted=grid_search.predict(x_test)

df_test=pd.DataFrame(list(zip(y_test,predicted)),columns=["real","predicted"])

k=pd.crosstab(df_test['real'],df_test["predicted"])
print(k)

predicted     0    1
real                
0          1781   26
1            55  169


In [47]:
TP=k.iloc[1,1]
TN=k.iloc[0,0]
FP=k.iloc[0,1]
FN=k.iloc[1,0]
P=TP+FN
N=TN+FP

print(TP,TN,FP,FN)
print('Accuracy is :',(TP+TN)/(P+N))
Sensitivity=TP/P
print('Sensitivity is :',Sensitivity)
print('Specificity is :',TN/N)
precision=TP/(TP+FP)
print('precision is :',precision)
print('f1 score is :',(2*Sensitivity*precision)/(Sensitivity+precision))

169 1781 26 55
Accuracy is : 0.9601181683899557
Sensitivity is : 0.7544642857142857
Specificity is : 0.9856115107913669
precision is : 0.8666666666666667
f1 score is : 0.8066825775656326


In [119]:
#3
parameters={
'learning_rate': ["invscaling"],
'hidden_layer_sizes': [(50,50,50), (50,100,50), (100,),(10,10,5),(100,600)],  #5 nodes, 10 nodes and 5 nodes of 3 hidden layers
'alpha': [0.0001, 0.05,0.0002],
'activation': [ "relu",'logistic',"tanh"],#'solver': ['sgd', 'adam'],
'learning_rate_init':[.01,0.02,0.002,0.03]
}
 

In [120]:
clf3=MLPClassifier()

In [121]:
grid_search=GridSearchCV(clf3,cv=10,param_grid=parameters,scoring='roc_auc',verbose=2,n_jobs=-1)

In [122]:
help(GridSearchCV)

Help on class GridSearchCV in module sklearn.model_selection._search:

class GridSearchCV(BaseSearchCV)
 |  GridSearchCV(estimator, param_grid, scoring=None, n_jobs=None, iid='warn', refit=True, cv='warn', verbose=0, pre_dispatch='2*n_jobs', error_score='raise-deprecating', return_train_score=False)
 |  
 |  Exhaustive search over specified parameter values for an estimator.
 |  
 |  Important members are fit, predict.
 |  
 |  GridSearchCV implements a "fit" and a "score" method.
 |  It also implements "predict", "predict_proba", "decision_function",
 |  "transform" and "inverse_transform" if they are implemented in the
 |  estimator used.
 |  
 |  The parameters of the estimator used to apply these methods are optimized
 |  by cross-validated grid-search over a parameter grid.
 |  
 |  Read more in the :ref:`User Guide <grid_search>`.
 |  
 |  Parameters
 |  ----------
 |  estimator : estimator object.
 |      This is assumed to implement the scikit-learn estimator interface.
 |     

In [123]:
grid_search.fit(x_train,y_train)

Fitting 10 folds for each of 180 candidates, totalling 1800 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   57.7s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed: 13.0min
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed: 32.3min
[Parallel(n_jobs=-1)]: Done 1005 tasks      | elapsed: 45.4min
[Parallel(n_jobs=-1)]: Done 1450 tasks      | elapsed: 57.6min
[Parallel(n_jobs=-1)]: Done 1800 out of 1800 | elapsed: 67.3min finished


GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=MLPClassifier(activation='relu', alpha=0.0001,
                                     batch_size='auto', beta_1=0.9,
                                     beta_2=0.999, early_stopping=False,
                                     epsilon=1e-08, hidden_layer_sizes=(100,),
                                     learning_rate='constant',
                                     learning_rate_init=0.001, max_iter=200,
                                     momentum=0.9, n_iter_no_change=10,
                                     nesterovs_momentum=True, power_t=0.5,
                                     random_st...
                                     warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid={'activation': ['relu', 'logistic', 'tanh'],
                         'alpha': [0.0001, 0.05, 0.0002],
                         'hidden_layer_sizes': [(50, 50, 50), (50, 100, 50),
                        

In [124]:
grid_search.best_params_

{'activation': 'relu',
 'alpha': 0.0002,
 'hidden_layer_sizes': (100,),
 'learning_rate': 'invscaling',
 'learning_rate_init': 0.02}

In [125]:
grid_search.best_estimator_

MLPClassifier(activation='relu', alpha=0.0002, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='invscaling',
              learning_rate_init=0.02, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=None, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [126]:
mlp3=grid_search.best_estimator_

In [127]:
mlp2.fit(x_train,y_train)

MLPClassifier(activation='logistic', alpha=0.1, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(20, 10), learning_rate='invscaling',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=None, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [128]:
predicted=grid_search.predict(x_test)

df_test=pd.DataFrame(list(zip(y_test,predicted)),columns=["real","predicted"])

k=pd.crosstab(df_test['real'],df_test["predicted"])
print(k)

predicted     0    1
real                
0          1800    7
1            31  193


In [129]:
TP=k.iloc[1,1]
TN=k.iloc[0,0]
FP=k.iloc[0,1]
FN=k.iloc[1,0]
P=TP+FN
N=TN+FP

print(TP,TN,FP,FN)
print('Accuracy is :',(TP+TN)/(P+N))
Sensitivity=TP/P
print('Sensitivity is :',Sensitivity)
print('Specificity is :',TN/N)
precision=TP/(TP+FP)
print('precision is :',precision)
print('f1 score is :',(2*Sensitivity*precision)/(Sensitivity+precision))

193 1800 7 31
Accuracy is : 0.9812900049236829
Sensitivity is : 0.8616071428571429
Specificity is : 0.9961261759822911
precision is : 0.965
f1 score is : 0.9103773584905661
